In [0]:
!pip install mip

     |████████████████████████████████| 11.8MB 2.5MB/s 


In [0]:
import math
import numpy as np
from numpy.linalg import inv
import pandas as pd
from scipy.optimize import linprog
import itertools
from mip import Model, xsum, maximize, BINARY, INTEGER

import matplotlib.pyplot as plt
%matplotlib inline

#import warnings
#warnings.filterwarnings('ignore')

#import io
#from google.colab import files

Using Python-MIP package version 1.7.2


In [0]:
class Macros():
    def __init__(self, carbs, fat, protein, calories=None, name=None):
      self.name = name
      # If macros are supplied in grams as integers, calculate calories from macros (any calories argument will be ignored)
      if type(carbs) == int and type(fat) == int and type(protein) == int:
        self.carbs = carbs
        self.fat = fat
        self.protein = protein
        self.calories = int(round(self.carbs*4 + self.fat*9 + self.protein*4))
      # If macros are supplied in percentages as floats, calculate macros from calories
      elif type(carbs) == float and carbs >= 0 and carbs <= 1\
          and type(fat) == float and fat >= 0 and fat <= 1\
          and type(protein) == float and protein >= 0 and protein <= 1\
          and round(carbs+fat+protein, 3) == 1.0 and not pd.isnull(calories):
        self.carbs = round(calories*carbs/4, 1)
        self.fat = round(calories*fat/9, 1)
        self.protein = round(calories*protein/4, 1)
        self.calories = int(round(calories))
      # Otherwise, raise an error
      else:
        raise ValueError('Macros must be specified:\n\
        In grams as integers (in which case the calories argument is ignored), or\n\
        In percentages as floats (which must sum to 1.0), and calories must also be supplied')

    @property
    def carbs(self):
      return self._carbs
    @carbs.setter
    def carbs(self, carbs):
      if carbs < 0:
        raise ValueError('Carbs cannot be negative')
      self._carbs = carbs

    @property
    def fat(self):
      return self._fat
    @fat.setter
    def fat(self, fat):
      if fat < 0:
        raise ValueError('Fat cannot be negative')
      self._fat = fat

    @property
    def protein(self):
      return self._protein
    @protein.setter
    def protein(self, protein):
      if protein < 0:
        raise ValueError('Protein cannot be negative')
      self._protein = protein

    def __add__(self, other):
      output = Macros(carbs = 1, fat = 1, protein = 1, calories = np.nan)
      output.carbs = round(self.carbs + other.carbs, 1)
      output.fat = round(self.fat + other.fat, 1)
      output.protein = round(self.protein + other.protein, 1)
      output.calories = int(round(self.calories  +other.calories))
      return output

    def __sub__(self, other):
      output = Macros(carbs = 1, fat = 1, protein = 1, calories = np.nan)
      output.carbs = round(self.carbs - other.carbs, 1)
      output.fat = round(self.fat - other.fat, 1)
      output.protein = round(self.protein - other.protein, 1)
      output.calories = int(round(self.calories - other.calories))
      return output

    def __mul__(self, other):
      output = Macros(carbs = 1, fat = 1, protein = 1, calories = np.nan)
      output.carbs = round(self.carbs*other, 1)
      output.fat = round(self.fat*other, 1)
      output.protein = round(self.protein*other, 1)
      output.calories = int(round(self.calories*other))
      return output
    __rmul__ = __mul__

    # True division assumes that other is a scalar, and returns a Macros object
    def __truediv__(self, other):
      output = Macros(carbs = 1, fat = 1, protein = 1, calories = np.nan)
      output.carbs = round(self.carbs/other, 1)
      output.fat = round(self.fat/other, 1)
      output.protein = round(self.protein/other, 1)
      output.calories = int(round(self.calories/other))
      return output

    # Floor division assumes that other is a Macros object, and returns an integer
    def __floordiv__(self, other):
      return int(min(self.carbs//max(other.carbs, 1e-4), self.fat//max(other.fat, 1e-4), self.protein//max(other.protein, 1e-4)))

    def __eq__(self, other):
      if self.carbs == other.carbs and self.fat == other.fat and self.protein == other.protein and self.calories == other.calories: 
        return True
      else:
        return False

    def __ne__(self, other):
      if self.carbs != other.carbs or self.fat != other.fat or self.protein != other.protein or self.calories != other.calories: 
        return True
      else:
        return False

    def __le__(self, other):
      if self.carbs <= other.carbs and self.fat <= other.fat and self.protein <= other.protein and self.calories <= other.calories: 
        return True
      else:
        return False

    def __ge__(self, other):
      if self.carbs >= other.carbs and self.fat >= other.fat and self.protein >= other.protein and self.calories >= other.calories: 
        return True
      else:
        return False

    def __repr__(self):
      if self.name is None:
        return str(self.carbs) + ' grams of carbs, ' + str(self.fat) +\
               ' grams of fat, ' + str(self.protein) + ' grams of protein, and ' + str(self.calories) + ' calories.'
      else:
        return str(self.name).capitalize() + ': ' + str(self.carbs) + ' grams of carbs, ' + str(self.fat) +\
               ' grams of fat, ' + str(self.protein) + ' grams of protein, and ' + str(self.calories) + ' calories.'

In [0]:
class Food(Macros):
    def __init__(self, carbs, fat, protein, calories=None, name='', serving=''):
      Macros.__init__(self, carbs, fat, protein, calories, name)
      self.serving = serving

    @property
    def calories(self):
      return self._calories
    @calories.setter
    def calories(self, calories):
      if calories < 5:
          raise ValueError('Calories must be at least 5')
      self._calories = calories

In [0]:
broccoli = Food(name = 'Broccoli', carbs = .70, fat = 0.0, protein = .30, calories = 250)
cabbage = Food(75, 0, 25, serving = '1 head')

In [0]:
broccoli*2

87.6 grams of carbs, 0.0 grams of fat, 37.6 grams of protein, and 500 calories.

In [0]:
broccoli*2-cabbage

12.6 grams of carbs, 0.0 grams of fat, 12.6 grams of protein, and 100 calories.

In [0]:
(broccoli+cabbage)*2-cabbage-cabbage-broccoli == broccoli

True

In [0]:
(broccoli+cabbage)*2-cabbage*1.9-0.1*cabbage-broccoli == broccoli

True

In [0]:
broccoli = Food(.70, 0.0, .30, calories = 250, serving = '1 crown', name = 'Broccoli')
cabbage = Food(.75, 0.0, .25, calories = 400, serving = '1 head', name = 'Cabbage')
onions = Food(.95, 0.0, .05, calories = 150, serving = '1 onion', name = 'Onions')
potatoes = Food(1.0, 0.0, 0.0, calories = 250, serving = '1 potato', name = 'Potatoes')

bananas = Food(1.0, 0.0, 0.0, calories = 100, serving = '1 banana', name = 'Bananas')
apples = Food(.90, 0.0, .10, calories = 90, serving = '1 apple', name = 'Apples')
blueberries = Food(.80, 0.0, .20, calories = 75, serving = '1 cup', name = 'Blueberries')
avocados = Food(.25, .65, .10, calories = 270, serving = '1/2 avocado', name = 'Avocados')

white_rice = Food(1.0, 0.0, 0.0, calories = 220, serving = '1 cup cooked', name = 'White rice')
oats = Food(.80, .10, .10, calories = 170, serving = '1 cup cooked', name = 'Oats')
white_bread = Food(.95, 0.0, .05, calories = 160, serving = '2 slices', name = 'White bread')
multigrain_bread = Food(.70, .15, .15, calories = 220, serving = '2 slices', name = 'Multigrain bread')

peanuts = Food(.15, .65, .20, calories = 140, serving = '1/4 cup', name = 'Peanuts')
almonds = Food(.25, .55, .20, calories = 100, serving = '1/4 cup', name = 'Almonds')
walnuts = Food(.20, .65, .15, calories = 120, serving = '1/4 cup', name = 'Walnuts')
macadamia_nuts = Food(.10, .80, .10, calories = 160, serving = '1/4 cup', name = 'Macadamia nuts')

skim_milk = Food(.60, 0.0, .40, calories = 90, serving = '1 cup', name = 'Skim milk')
butter = Food(0.0, 1.0, 0.0, calories = 100, serving = '1 Tbsp', name = 'Butter')
cheddar_cheese = Food(.10, .55, .35, calories = 100, serving = '1/2 cup shredded', name = 'Cheddar cheese')
cottage_cheese = Food(.10, .25, .65, calories = 200, serving = '1 cup', name = 'Cottage cheese (lowfat)')

ground_beef = Food(0.0, .55, .45, calories = 160, serving = '4 oz', name = 'Ground beef (80% lean)')
chicken_breast = Food(0.0, .05, .95, calories = 110, serving = '4 oz', name = 'Chicken breast (skinless)')
bacon = Food(0.0, .80, .20, calories = 90, serving = '3 slices', name = 'Bacon')
eggs = Food(0.0, .40, .60, calories = 80, serving = '2 eggs', name = 'Eggs')

salmon = Food(0.0, .25, .75, calories = 130, serving = '4 oz', name = 'Salmon (wild, with skin)')
cod = Food(0.0, .05, .95, calories = 110, serving = '4 oz', name = 'Cod (wild, without skin)')
shrimp = Food(0.0, 0.0, 1.0, calories = 100, serving = '4 oz', name = 'Shrimp')
sardines = Food(0.0, .30, .70, calories = 160, serving = '1 can', name = 'Sardines')

potato_chips = Food(.60, .40, 0.0, calories = 320, serving = '2 oz', name = 'Potato chips')
candy_bar = Food(.70, .20, .10, calories = 320, serving = '1 bar', name = 'Candy bar')
pepperoni_pizza = Food(.30, .45, .25, calories = 450, serving = '1 slice', name = 'Pepperoni pizza')
ice_cream = Food(.50, .40, .10, calories = 170, serving = '1/2 cup', name = 'Ice cream')

In [0]:
library = [broccoli, cabbage, onions, potatoes, bananas, apples, blueberries,
           avocados, white_rice, oats, white_bread, multigrain_bread, peanuts,
           almonds, walnuts, macadamia_nuts, skim_milk, butter, cheddar_cheese,
           cottage_cheese, ground_beef, chicken_breast,
           bacon, eggs, salmon, cod, shrimp, sardines,
           potato_chips, candy_bar, pepperoni_pizza, ice_cream]

In [0]:
library[0]

Broccoli: 43.8 grams of carbs, 0.0 grams of fat, 18.8 grams of protein, and 250 calories.

In [0]:
def unique(food_library):
    '''
    INPUT:
    food_library - a list of food items, each of which is a Macros object (usually a Food object)

    STEPS:
    for each food, add to the list of unique foods if it is not already there

    OUTPUT:
    a list of unique food items, each of which is a Macros object (usually a Food object)
    '''
    unique_food_library = []
    for food in food_library:
      if food not in unique_food_library:
        unique_food_library.append(food)
    return unique_food_library

In [0]:
unique([oats, peanuts, cod, cod, eggs])

[Oats: 34.0 grams of carbs, 1.9 grams of fat, 4.2 grams of protein, and 170 calories.,
 Peanuts: 5.2 grams of carbs, 10.1 grams of fat, 7.0 grams of protein, and 140 calories.,
 Cod (wild, without skin): 0.0 grams of carbs, 0.6 grams of fat, 26.1 grams of protein, and 110 calories.,
 Eggs: 0.0 grams of carbs, 3.6 grams of fat, 12.0 grams of protein, and 80 calories.]

In [0]:
def calc_macros(food_library, solution):
    '''
    INPUT:
    food_library - a list of Macros or Food objects containing the foods from which to draw possible combinations
    solution - a list of numbers

    STEPS:
    For a list of Macros objects (food_library), calculate the sum weighted by a list of numbers (solution)

    OUTPUT:
    a Macros object that contains the weighted sum
    '''
    if len(food_library) > 0 and len(food_library) == len(solution):
      macros = solution[0]*food_library[0]
      for i in range(1, len(solution)):
        macros += solution[i]*food_library[i]
      return macros
    else:
      raise ValueError('food_library and solution must be non-empty and have the same length')

In [0]:
calc_macros(food_library, [1, 1, 1, 2])

39.2 grams of carbs, 19.8 grams of fat, 61.3 grams of protein, and 580 calories.

In [0]:
def find_solution(food_library, goal):
    '''
    INPUT:
    food_library - a list of Macros or Food objects containing the foods from which to draw possible combinations
    goal - a Macros object within which food combinations must lie

    STEPS:
    Iterate through food objects in the order given in food_library. For each:
      Calculate the maximum number of servings that fit within the goal using floor division
      Reduce the goal by that many servings of that food

    OUTPUT:
    a list of integers that represents a possible combination of foods that fit within the goal,
    specifically the solution that maximizes the first food first, then the second, and so on
    '''
    solution = [0]*len(food_library)
    for i in range(len(food_library)):
      solution[i] = goal//food_library[i]
      goal -= solution[i]*food_library[i]
    return solution

In [0]:
find_solution(food_library, goal)

[5, 5, 3, 1]

In [0]:
def print_solution(food_library, solution):
    '''
    INPUT:
    food_library - a list of Macros or Food objects containing the foods from which to draw possible combinations
    solution - a list of numbers that represents a possible combination of foods

    STEPS:
    Print the number of servings of each food item
    Calculate the total macros and calories
    Print the total macros and calories

    OUTPUT:
    None
    '''
    output_str = ''
    # For each food, add a description of how many servings are present in the solution
    for i in range(len(food_library)):
      if solution[i] == 1:
        output_str = output_str+str(solution[i])+' serving of '+str(food_library[i].name)+', '
      else:
        output_str = output_str+str(solution[i])+' servings of '+str(food_library[i].name)+', '
    # Remove the final ', '
    output_str = output_str[:-2]
    # Print the description
    print(output_str)
    # Print the macros and calories
    print(calc_macros(food_library, solution))
    # Print blank line
    print()
    return None

In [0]:
print_solution(food_library, [5, 5, 3, 1])

5 servings of Oats, 5 servings of Peanuts, 3 servings of Cod (wild, without skin), 1 serving of Eggs
196.0 grams of carbs, 65.4 grams of fat, 146.3 grams of protein, and 1960 calories.



In [0]:
def find_next_solution(food_library, goal, previous_solution):
    '''
    INPUT:
    food_library - a list of Macros or Food objects containing the foods from which to draw possible combinations
    goal - a Macros object within which food combinations must lie
    previous_solution - a list of integers that represents a possible combination of foods that fit within the goal

    STEPS:
    Remove the last element of previous_solution
    Of what remains, find the last non-zero element and reduce it by 1
    Set the resulting list of numbers as the beginning of the next solution
    To find the remaining part of the next solution:
      Reduce the goal by the existing part of the solution
      Pass a subset of the food library and the reduced goal to find_solution()

    OUTPUT:
    a list of integers that represents the solution that comes next in the order of solutions where
    the first food is maximized first, then the second, and so on
    '''
    # Chop off last element
    goal_reduction_amount = previous_solution[:-1]
    
    # If all but the last element of the previous solution are 0, there is no next solution
    if max(goal_reduction_amount) == 0:
      return None
    else:
      # Find last non-zero element, reduce it by 1
      found = False
      while not found:
        if goal_reduction_amount[-1] > 0:
          goal_reduction_amount[-1] -= 1
          found = True
        else:
          goal_reduction_amount = goal_reduction_amount[:-1]

      # Reduce goal by goal_reduction_amount
      goal -= calc_macros(food_library[:len(goal_reduction_amount)], goal_reduction_amount)
      
      # Remove first n elements of food_library, where n = len(goal_reduction_amount)
      food_library = food_library[len(goal_reduction_amount):]

      # Calculate next solution
      solution = find_solution(food_library, goal)
      goal_reduction_amount.extend(solution)
      return goal_reduction_amount

In [0]:
find_next_solution(food_library, goal, [5, 5, 3, 1])

[5, 5, 2, 1]

In [0]:
def pareto_optimal(solution, ensemble):
    '''
    INPUT:
    solution - a list of integers that represents a possible combination of foods that fit within the goal
    ensemble - a list of lists including all pareto optimal solutions found so far

    STEPS:
    if the solution is of type None (which will be the output of find_next_solution() when no more solutions exist),
    return False, thereby not allowing the solution to join the ensemble
    else, check the solution against every solution in the ensemble
    if any solution in the ensemble is greater than or equal to the solution for all list indices, the solution is not pareto optimal
    otherwise, it is pareto optimal

    OUTPUT:
    a boolean representing whether the solution is pareto optimal to the ensemble of existing solutions
    '''
    if solution is None:
      return False
    else:
      solution = np.array(solution)
      ensemble = np.array(ensemble)
      for entry in ensemble:
        if min(entry-solution) >= 0:
          return False
      return True

In [0]:
pareto_optimal([5, 5, 2, 1], [[5, 5, 3, 1]])

False

In [0]:
def calc_pareto_ensemble(food_library, goal, print_output=False):
    '''
    INPUT:
    food_library - a list of Macros or Food objects containing the foods from which to draw possible combinations
    goal - a Macros object within which food combinations must lie
    print_output - whether to print pareto optimal solutions as they are found, default is False

    STEPS:
    Remove foods from food library that are repeats or for which one serving does not fit within goal
    Find first solution, create ensemble
    Find more solutions, add to ensemble if they are pareto optimal
    If print_output is True, print pareto optimal solutions as they are found

    OUTPUT:
    a list of lists including all pareto optimal solutions, in descending order
    of number of servings of first food, then second, and so on
    '''
    # Keep only unique instances of food items
    food_library = unique(food_library)
    # Keep only those foods for which at least one serving fits within goal
    food_library = [food for food in food_library if goal//food > 0]
    # Find first solution
    solution = find_solution(food_library, goal)
    ensemble = [solution]
    if print_output:
      print_solution(food_library, solution)
    # Find all other solutions
    while solution is not None:
      solution = find_next_solution(food_library, goal, solution)
      if pareto_optimal(solution, ensemble):
        ensemble.append(solution)
        if print_output:
          print_solution(food_library, solution)
    # Return ensemble
    return ensemble

In [0]:
goal = Macros(.40, .30, .30, calories = 2000)
goal

200.0 grams of carbs, 66.7 grams of fat, 150.0 grams of protein, and 2000 calories.

In [0]:
food_library = [oats, peanuts, cod, eggs]
ensemble = calc_pareto_ensemble(food_library, goal, print_output=True)

5 servings of Oats, 5 servings of Peanuts, 3 servings of Cod (wild, without skin), 1 serving of Eggs
196.0 grams of carbs, 65.4 grams of fat, 146.3 grams of protein, and 1960 calories.

5 servings of Oats, 4 servings of Peanuts, 2 servings of Cod (wild, without skin), 4 servings of Eggs
190.8 grams of carbs, 65.5 grams of fat, 149.2 grams of protein, and 1950 calories.

5 servings of Oats, 3 servings of Peanuts, 4 servings of Cod (wild, without skin), 0 servings of Eggs
185.6 grams of carbs, 42.2 grams of fat, 146.4 grams of protein, and 1710 calories.

5 servings of Oats, 3 servings of Peanuts, 3 servings of Cod (wild, without skin), 2 servings of Eggs
185.6 grams of carbs, 48.8 grams of fat, 144.3 grams of protein, and 1760 calories.

5 servings of Oats, 3 servings of Peanuts, 1 serving of Cod (wild, without skin), 6 servings of Eggs
185.6 grams of carbs, 62.0 grams of fat, 140.1 grams of protein, and 1860 calories.

5 servings of Oats, 3 servings of Peanuts, 0 servings of Cod (wild,

To print the servings and macros for an existing ensemble:

In [0]:
for solution in ensemble:
  print_solution(food_library, solution)

5 servings of Oats, 5 servings of Peanuts, 3 servings of Cod (wild, without skin), 1 serving of Eggs
196.0 grams of carbs, 65.4 grams of fat, 146.3 grams of protein, and 1960 calories.

5 servings of Oats, 4 servings of Peanuts, 2 servings of Cod (wild, without skin), 4 servings of Eggs
190.8 grams of carbs, 65.5 grams of fat, 149.2 grams of protein, and 1950 calories.

5 servings of Oats, 3 servings of Peanuts, 4 servings of Cod (wild, without skin), 0 servings of Eggs
185.6 grams of carbs, 42.2 grams of fat, 146.4 grams of protein, and 1710 calories.

5 servings of Oats, 3 servings of Peanuts, 3 servings of Cod (wild, without skin), 2 servings of Eggs
185.6 grams of carbs, 48.8 grams of fat, 144.3 grams of protein, and 1760 calories.

5 servings of Oats, 3 servings of Peanuts, 1 serving of Cod (wild, without skin), 6 servings of Eggs
185.6 grams of carbs, 62.0 grams of fat, 140.1 grams of protein, and 1860 calories.

5 servings of Oats, 3 servings of Peanuts, 0 servings of Cod (wild,

The following test case checks if the same ensemble is created regardless of the order in which food items are specified in the library.

In [0]:
# Build ensemble from a library of oats, peanuts, cod, and eggs
ensemble1 = calc_pareto_ensemble([oats, peanuts, cod, eggs], goal)
ensemble1[:10]

[[5, 5, 3, 1],
 [5, 4, 2, 4],
 [5, 3, 4, 0],
 [5, 3, 3, 2],
 [5, 3, 1, 6],
 [5, 3, 0, 7],
 [5, 2, 3, 3],
 [5, 2, 2, 5],
 [5, 2, 1, 7],
 [5, 2, 0, 9]]

In [0]:
# The ensemble is already sorted in descending order (by the nature of how it was created)
ensemble1 == sorted(ensemble1, reverse=True)

True

In [0]:
# Build ensemble from a library of cod, oats, eggs, and peanuts
ensemble2 = calc_pareto_ensemble([cod, oats, eggs, peanuts], goal)
ensemble2[:10]

[[5, 4, 0, 0],
 [5, 2, 0, 1],
 [5, 1, 1, 0],
 [5, 1, 0, 2],
 [5, 0, 1, 1],
 [4, 5, 2, 0],
 [4, 5, 1, 1],
 [4, 5, 0, 3],
 [4, 4, 1, 2],
 [4, 4, 0, 4]]

In [0]:
# Rearrange each solution to be in order of oats, peanuts, cod, and eggs
ensemble2_rearranged = []
for solution in ensemble2:
  ensemble2_rearranged.append([solution[1], solution[3], solution[0], solution[2]])
ensemble2_rearranged[:10]

[[4, 0, 5, 0],
 [2, 1, 5, 0],
 [1, 0, 5, 1],
 [1, 2, 5, 0],
 [0, 1, 5, 1],
 [5, 0, 4, 2],
 [5, 1, 4, 1],
 [5, 3, 4, 0],
 [4, 2, 4, 1],
 [4, 4, 4, 0]]

In [0]:
# Sort descending
ensemble2_rearranged = sorted(ensemble2_rearranged, reverse=True)
ensemble2_rearranged[:10]

[[5, 5, 3, 1],
 [5, 4, 2, 4],
 [5, 3, 4, 0],
 [5, 3, 3, 2],
 [5, 3, 1, 6],
 [5, 3, 0, 7],
 [5, 2, 3, 3],
 [5, 2, 2, 5],
 [5, 2, 1, 7],
 [5, 2, 0, 9]]

In [0]:
# Check if the ensembles are identical
ensemble1 == ensemble2_rearranged

True

In [0]:
def check_pareto(ensemble):
  is_pareto = [False]*len(ensemble)
  for i, solution in enumerate(ensemble):
    tmp = ensemble.copy()
    tmp.remove(solution)
    is_pareto[i] = pareto_optimal(solution, tmp)
  return min(is_pareto)

In [0]:
check_pareto(ensemble)

# Notes

In [0]:
possible_servings = np.unique(possible_servings, axis=0)

In [0]:
for i, serving in enumerate(possible_servings):
  print('Option '+str(i+1)+':')
  print(str(serving[0])+' servings of '+str(library_[0].name)+', '+
        str(serving[1])+' servings of '+str(library_[1].name)+', '+
        str(serving[2])+' servings of '+str(library_[2].name)+', and '+
        str(serving[3])+' servings of '+str(library_[3].name))
  print(serving[0]*library_[0]+serving[1]*library_[1]+serving[2]*library_[2]+serving[3]*library_[3])
  print()

In [0]:
# Determine first solution
a = goal//food_library[0]
b = (goal-a*food_library[0])//food_library[1]
c = (goal-a*food_library[0]-b*food_library[1])//food_library[2]
ensemble = [[a, b, c]]
print(str(a)+' servings of '+food_library[0].name+', '+
      str(b)+' servings of '+food_library[1].name+', and '+
      str(c)+' servings of '+food_library[2].name)

while a > 0:
  a -= 1
  b = (goal-a*food_library[0])//food_library[1]
  c = (goal-a*food_library[0]-b*food_library[1])//food_library[2]
  if pareto_optimal([a, b, c], ensemble):
    ensemble.append([a, b, c])
    print(str(a)+' servings of '+food_library[0].name+', '+
          str(b)+' servings of '+food_library[1].name+', and '+
          str(c)+' servings of '+food_library[2].name)
  while b > 0:
    b -= 1
    c = (goal-a*food_library[0]-b*food_library[1])//food_library[2]
    if pareto_optimal([a, b, c], ensemble):
      ensemble.append([a, b, c])
      print(str(a)+' servings of '+food_library[0].name+', '+
            str(b)+' servings of '+food_library[1].name+', and '+
            str(c)+' servings of '+food_library[2].name)